# Chapter 8 - Dependency Injection

- DI is a way to apply the IoC principle
    * D is SOLID

## Composition root

- in .net 8+ the composition root is `Program.cs`
- We give the IoC container a bluebrint and it handles the objects for us
    * includes their lifetime

### Control freak code smell

- Forbids us from using the `new` keyword
- Violates the IoC principle
- Always ask if it is a dependcy that could be injected instead

### Stable dependencies

- unlikley to break something if they change i.e. DTO or other data structure with no behavior
- Can use `new` for these

### Volatile dependencies

- i.e. Data access, business logic
- Use DI

### Lifetime

|Lifetime   |Description                                            |
|-----------|-------------------------------------------------------|
| Transient | Creates a new instance every time it is requested     |
| Scoped    | Create 1 instance per HTTP request                    |
| Singleton | Creates a single instance for the entire application  |

- Make sure singleton's are thead safe

In [1]:
public class ObjectLifetime : ITransient, IScoped, ISingleton
{
    public Guid Id { get; } = Guid.NewGuid();
}

public interface ISingleton
{
    Guid Id { get; }
}

public interface IScoped
{
    Guid Id { get; }
}

public interface ITransient
{
    Guid Id { get; }
}

public class ServiceConsumer(ISingleton singleton, IScoped scoped, ITransient 
transient)
{
    private readonly ISingleton _singleton = singleton;
    private readonly IScoped _scoped = scoped;
    private readonly ITransient _transient = transient;
    public Guid SingletonId => _singleton.Id;
    public Guid ScopedId => _scoped.Id;
    public Guid TransientId => _transient.Id;
}

### References needed to run the app below

In [12]:
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.Hosting.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Mvc.ViewFeatures.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Diagnostics.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.Results.dll"

In [4]:
string[] args = {"--urls","http://localhost:5555"};

In [24]:
using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;

var builder = WebApplication.CreateBuilder(args);
builder.Services.AddSingleton<ISingleton, ObjectLifetime>();
builder.Services.AddScoped<IScoped, ObjectLifetime>();
builder.Services.AddTransient<ITransient, ObjectLifetime>();
builder.Services.AddTransient<ServiceConsumer>();
var app = builder.Build();
app.MapGet("/", (ServiceConsumer serviceConsumer1, ServiceConsumer serviceConsumer2) =>
{
    return TypedResults.Ok(new[] {
        serviceConsumer1,
        serviceConsumer2
    });
});
app.RunAsync();

info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:5555
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\jason\training\dotnet\Architecting-ASP.NET-Core-Applications-3E\MyNotes


### Stop the server

In [26]:
app.StopAsync();
app = null;

In [25]:
GET http://localhost:5555

Name,Value
traceparent,00-5f57055280abc71c1db3f3a955c7f0ce-d80405514749824c-00
Name,Value
Date,"Wed, 07 May 2025 20:47:23 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,317
singletonId,"""1a3cbbda-54fd-4675-9fa1-1004d23e0a71"""
scopedId,"""7c03ce28-f2f5-405c-b287-bde8cd9a95f5"""
transientId,"""489b3ca9-ee76-46cc-9d62-403b49335e8a"""


### Register with DI Elegantly

- Use extension method to register

    ```csharp
    namespace Microsoft.Extensions.DependencyInjection;
    public static class DemoFeatureExtensions
    {
        public static IServiceCollection AddDemoFeature(this IServiceCollection services)
        {
            return services
                .AddSingleton<MyFeature>()
                .AddSingleton<IMyFeatureDependency, MyFeatureDependency>()
            ;
        }
    }
    ```

    Then in `Program.cs` we can call:

    ```csharp
    builder.Services.AddDemoFeature();
    ```

### Other options

- Try to always use the built in IoC but there are other libraries or you can create your own if need be see p. 247

## Strategy Pattern Revisited

### Injection methods

- Constructor (Most common)
- Property (Not supported out of the box)
    * Consider a different design instead of this
- Method
    * optional dependencies

Use DI to inject different services at runtime using an Interface

ie.

```csharp
public class MyController : ControllerBase {
    public MyController( ILocationService locationService){}
}
```

Program.cs

https://andrewlock.net/configuring-environment-specific-services-in-asp-net-core-part-2/

```csharp
if(builder.Environment.IsDevelopment())
{
    builder.Services.AddSingleton<ILocationService, InMemoryLocationService>();
}
else {
    builder.Services.AddSingleton<ILocationService, SQLServerLocationService>();
}
```

## Singleton Patter Revistited

- Classic ASP had an application state
    * Dictionary of key/value pairs shared for all requests
    * Does not exist in .NET Core

### Example of caching in memory 

- Only works per process - Use `IDistributedCache` for distributed caching to something like Redis
    

In [ ]:
using Microsoft.Extensions.Caching.Memory;

public interface IApplicationState
{
    TItem? Get<TItem>(string key);
    bool Has<TItem>(string key);
    void Set<TItem>(string key, TItem value) where TItem : notnull;
}

public class ApplicationMemoryCache : IApplicationState
{
 private readonly IMemoryCache _memoryCache;
 public ApplicationMemoryCache(IMemoryCache memoryCache)
 {
 _memoryCache = memoryCache ?? throw new
ArgumentNullException(nameof(memoryCache));
 }
 public TItem Get<TItem>(string key)
 {
 return _memoryCache.Get<TItem>(key);
 }
 public bool Has<TItem>(string key)
 {
 return _memoryCache.TryGetValue<TItem>(key, out _);
 }
 public void Set<TItem>(string key, TItem value)
 {
 _memoryCache.Set(key, value);
 }
}

### Project - Wishlist

In [29]:
using System.Collections.Concurrent;

public interface IWishList
{
    Task<WishListItem> AddOrRefreshAsync(string itemName);
    Task<IEnumerable<WishListItem>> AllAsync();
}

public interface ISystemClock
{
    DateTimeOffset UtcNow { get; }
}

public class SystemClock : ISystemClock
{
    // Same as: public DateTimeOffset UtcNow => TimeProvider.System.GetUtcNow();
    public DateTimeOffset UtcNow => DateTimeOffset.UtcNow;
}

public record class WishListItem(string Name, int Count, DateTimeOffset Expiration);

public class InMemoryWishListOptions
{
    public ISystemClock SystemClock { get; set; } = new SystemClock();
    public int ExpirationInSeconds { get; set; } = 30;
}

public class InMemoryWishList : IWishList
{
    private readonly InMemoryWishListOptions _options;
    private readonly ConcurrentDictionary<string, InternalItem> _items = new();

    public InMemoryWishList(InMemoryWishListOptions options)
    {
        _options = options ?? throw new ArgumentNullException(nameof(options));
    }

    public Task<WishListItem> AddOrRefreshAsync(string itemName)
    {
        var expirationTime = _options.SystemClock.UtcNow.AddSeconds(_options.ExpirationInSeconds);
        _items
            .Where(x => x.Value.Expiration < _options.SystemClock.UtcNow)
            .Select(x => x.Key)
            .ToList()
            .ForEach(key => _items.Remove(key, out _))
        ;
        var item = _items.AddOrUpdate(
            itemName,
            new InternalItem(Count: 1,Expiration: expirationTime),
            (string key, InternalItem item) => item with {
                Count = item.Count + 1,
                Expiration = expirationTime
            }
        );
        var wishlistItem = new WishListItem(
            Name: itemName,
            Count: item.Count,
            Expiration: item.Expiration
        );
        return Task.FromResult(wishlistItem);
    }

    public Task<IEnumerable<WishListItem>> AllAsync()
    {
        var items = _items
            .Where(x => x.Value.Expiration >= _options.SystemClock.UtcNow)
            .Select(x => new WishListItem(
                Name: x.Key,
                Count: x.Value.Count,
                Expiration: x.Value.Expiration
            ))
            .OrderByDescending(x => x.Count)
            .AsEnumerable()
        ;
        return Task.FromResult(items);
    }

    private record class InternalItem(int Count, DateTimeOffset Expiration);
}


In [30]:
string[] args = {"--urls","http://localhost:7001"};

In [31]:
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.Hosting.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Mvc.ViewFeatures.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Diagnostics.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.Results.dll"

In [33]:
using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;

var builder = WebApplication.CreateBuilder(args);
builder.Services
 .AddSingleton<InMemoryWishListOptions>()
 .AddSingleton<IWishList, InMemoryWishList>();

var app = builder.Build();

app.MapGet("/", async (IWishList wishList) => await wishList.AllAsync());

app.MapPost("/", async (IWishList wishList, CreateItem? newItem) =>
{
    if (newItem?.Name == null)
    {
        return Results.BadRequest();
    }
    var item = await wishList.AddOrRefreshAsync(newItem.Name);
    return Results.Created("/", item);
});

app.RunAsync();

public record class CreateItem(string? Name);

info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:7001
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\jason\training\dotnet\Architecting-ASP.NET-Core-Applications-3E\MyNotes


In [ ]:
GET http://localhost:7001/

In [ ]:
POST http://localhost:7001/
Content-Type: application/json

{
    "name": "Item 1"
}

In [42]:
app.StopAsync();
app = null;

See complete project at `C08\Wishlist` and tests at `C08\Wishlist.Tests`

## Guard Clauses

- No gaurauntess that injected classes are not null

    instead of:

    ```csharp
    _locationService = locationService;
    ```

    ```csharp
    locationService = locationService ?? throw new ArgumentNullException(nameof(locationService));
    ```

- The built-in DI container will throw an exception if can't fulfill all dependencies during the instanctiation
    * Author recommends adding gaurds regardless

- Helper

```
ArgumentNullException.ThrowIfNull(locationService)
```

## Service Locator Pattern

- Anti-pattern or code smell
- Should not be needed in most cases

```
app.Services.CreateScope().ServiceProvider.GetRequiredService<ApplicationDBContext>()
```

```
IServiceProvider
```

## Facotry Pattern

Can you pre-processing directive to direct the compiler

https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/preprocessor-directives

Delegate acts as a factory. A different way to achieve above.

```csharp
builder.Services.AddSingleton<ILocationService>(sp =>
{
    if (builder.Environment.IsDevelopment())
    {
        return new InMemoryLocationService();
    }
    return new SqlLocationService(new NotImplementedDatabase());
}
```

